In [2]:
import cv2
from skimage.transform import resize
from os import listdir, walk
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

def gray_reshape(directory, new_size = (28,28), gray =False):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized  =[]
    pictures = []
    labels = []
    mapping = {'Agaricus' : 0,'Amanita' : 1,'Boletus' : 2,'Cortinarius' : 3,'Entoloma' : 4,'Hygrocybe' : 5,'Lactarius' : 6,'Russula' : 7,'Suillus' : 8, 
                'nail_polishes' : 9  ,'shoes' : 10 
                }
    onlyfolders = [f for f in listdir(directory)]
    for idx, folder in enumerate(onlyfolders[1:]):
        pictures = [f for f in listdir(directory + folder) if isfile(join(directory + folder , f))]
        for idx, picture in enumerate(pictures):
            item = cv2.imread(directory + folder +'/'+ picture )
            labels.append(mapping[folder])
            
            if gray == True:
                item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
                item_resized_gray = resize(item_gray, new_size)
                items_resized.append(item_resized_gray)
            else:
                item_resized_colored = resize(item, new_size)
                items_resized.append(item_resized_colored)
    return items_resized, labels

In [8]:
test0 = pd.read_csv('data/Data/fungiclef2022/FungiCLEF2022_test_metadata.csv',sep = ',',header = None)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (1,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
test0.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,ObservationId,eventDate,month,day,countryCode,Location_lvl0,Location_lvl3,Location_lvl2,Location_lvl1,Substrate,Habitat,filename
1,0.0,3305985310,2021-06-15T00:00:00,6.0,15.0,DK,Ellesøen,Denmark,Sjælland,Køge,dead wood (including bark),Bog woodland,0-3305985310.JPG
2,1.0,3305985310,2021-06-15T00:00:00,6.0,15.0,DK,Ellesøen,Denmark,Sjælland,Køge,dead wood (including bark),Bog woodland,1-3305985310.JPG


In [23]:
test1 = pd.read_csv('data/Data/fungiclef2022/DF20-train_metadata.csv',sep = ',')

In [32]:
test1.head(3)
test1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266344 entries, 0 to 266343
Data columns (total 34 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gbifID                266344 non-null  int64  
 1   eventDate             266328 non-null  object 
 2   year                  266328 non-null  float64
 3   month                 266328 non-null  float64
 4   day                   266328 non-null  float64
 5   countryCode           266344 non-null  object 
 6   locality              266273 non-null  object 
 7   taxonID               266344 non-null  float64
 8   scientificName        266344 non-null  object 
 9   kingdom               266344 non-null  object 
 10  phylum                266344 non-null  object 
 11  class                 266315 non-null  object 
 12  order                 266187 non-null  object 
 13  family                263975 non-null  object 
 14  genus                 266344 non-null  object 
 15  

In [28]:
mash = list(test1.scientificName.unique())

In [12]:
test2 = pd.read_csv('data/Data/fungiclef2022/DF20-val_metadata.csv',sep = ',',header = None)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (0,2,3,4,7,16,28,29,30,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
test2.head(2)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,class_id
1,2238478820,2016-10-12T00:00:00,2016.0,10.0,12.0,DK,Freerslev Hegn,44101.0,Lactarius necator (Bull.) Pers.,Fungi,...,Hillerød,2238478820-167801,soil,Jesper Ruf Larsen,55.920101,12.231723,25.0,coniferous woodland/plantation,2238478820-167801.JPG,779
